In [3]:
import torch
import numpy as np
import pickle
#from torchsummary import summary
from collections import OrderedDict
#from torch.utils.tensorboard import SummaryWriter
import datetime
import copy

In [4]:
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        # parameters
        self.depth = len(layers)-1
        #print(layers)
        # set up layer order dict
        self.activation = torch.nn.ReLU
        
        layer_list = list()
        for i in range(self.depth-1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        #print(layers[-2])
        #print(layers[-1])
        
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [94]:
## Asssume the dim of Traing and Testing are in shape [N,C,H,W]

class DE_MLP():
    def __init__(self, outdim=1,maxdepth=70,mindepth=5,minneuron=4,maxneuron=10,bsize=10,epoch=100,initSize=20,maxiter=10,stopcount=3,\
                 trainingset=None,validationset=None,trainingTarget=None,validateTarget=None):
        self.best=[]
        self.mean=[]
        self.outdim=outdim
        self.maxdepth=maxdepth
        self.mindepth=mindepth
        self.minneuron = minneuron
        self.maxneuron = maxneuron
        self.bszie = bsize
        self.epoch = epoch
        self.stopcount = stopcount
        self.pplSize = initSize
        self.maxiter = maxiter
        self.training = trainingset.reshape((trainingset.shape[0],-1))
        self.validationSet = validationset.reshape((validationset.shape[0],-1))
        self.target=trainingTarget
        self.validationTarget = validateTarget
        self.MLPlayerlist = []
        self.depthlist=np.random.choice(range(self.mindepth,self.maxdepth),self.pplSize,replace=False)
        
        if torch.cuda.is_available():
            self.device = torch.device('cuda')
        else:
            self.device = torch.device('cpu')

        for i in range(self.pplSize):
            depth = self.depthlist[i]
            tmp=[]
            tmp.append(self.training.shape[1])
            for j in range(depth):
                tmp.append(np.random.choice(range(self.minneuron,self.maxneuron),1,replace=False)[0])
            tmp.append(self.outdim)
            tmp=np.array(tmp)
            self.MLPlayerlist.append(tmp)
        
    def fit(config,id_):
        dnn = DNN(config)
        dnn.layers.to(self.device)
        best = float('inf')
        stop=0
        self.optimizer(dnn.layers.parameters(), lr=0.001)
        loss = torch.nn.BCEWithLogitsLoss()
        batch = self.trainingset.shape[0]//self.bsize
        vbatch = self.validationSet.shape[0]//self.bsize
        idxs = [x for x in range(self.trainingset.shape[0])]
        vidxs = [x for x in range(self.validationSet.shape[0])]
        for e in range(self.epoch):
            np.random.shuffle(idxs)
            dnn.layers.train()
            batchloss=0
            for i in range(batch):
                idx=idxs[i*self.bsize:i*self.bsize+self.bsize]
                self.optimizer.zero_grad()
                data = torch.tensor(self.trainingset[idx]).float().to(self.device)
                y = torch.tensor(self.target[idx])
                yhat = dnn(data)
                l = loss(yhat,y)
                batchloss+=l.item()
                loss.backward()
                self.optimizer.step()

            dnn.layers.eval()
            np.random.shuffle(vidxs)
            vloss=0
            for i in range(vbatch):
                vidx=vidxs[i*self.bsize:i*self.bsize+self.bsize]
                vdata = torch.tensor(self.validationSet[vidx]).float().to(self.device)
                vy = torch.tensor(self.target[vidx])
                vyhat = dnn(vdata)
                vl = loss(vyhat,vy)
                vloss += vl.item()
            if(vloss<best):
                vloss=best
            else:
                stop+=1
            print(f'ConfigID: {id_:3d}, Epoch: {e:3d}, Training Loss: {(batchloss/batch):10.8f}, Validation Loss: {(vloss/vbatch):10.8f},\
                    Best: {best:10.8f}, StopCount/Limit: {stop:3d}/{stopcount:3d}')

            if(stop>=self.stopcount):
                return best,config,id_ 

    def mutation_1_2_z(self,x1,xs,beta,debug=False):
        indim = x1[0]
        x1 = x1[1:-1] # remove in/out dim
        xs[0] = xs[0][1:-1]
        xs[1] = xs[1][1:-1]
        if(debug):
            print(f'M1 : x1 len {x1.shape[0]} xs0 len {xs[0].shape[0]} xs1 len {xs[1].shape[0]}')
            print(f'M1 : x1 {x1} \nM1 : xs0 {xs[0]} \nM1 : xs1 len {xs[1]}')
        # number of hidden layer mutation 
        # if the new length == 0 , set it back to target len , if <0 , take abs
        minlen = np.min([x1.shape[0],xs[0].shape[0],xs[1].shape[0]])
        if(debug):
            print(f'M1 : minlen {minlen}')          
        newminlen = minlen
        targetlen=int(np.floor((x1.shape[0]) + beta * (xs[0].shape[0] - xs[1].shape[0])))
        if(targetlen==0):
            targetlen=x1.shape[0]
        elif(targetlen<0):
            targetlen=abs(targetlen)
        if(targetlen < minlen):
            newminlen=targetlen
        if(debug):
            print(f'M1 : New Min Len :{newminlen}, Length Mutation :{targetlen}')
            
        #node mutation
        #As x1'len , xs' len and new len will be different, it will first do the node number mutation for min len, then apply the same rule to remaining if need. 
        xa = np.zeros((targetlen),dtype=int)
        # Node number mutation up to min len
        xa = x1[:newminlen] + beta * (xs[0][:newminlen] - xs[1][:newminlen]) # mutate on node with minlen
        # Node number mutation for the rest
        if(targetlen>minlen):
            xaa = np.zeros((targetlen-minlen))
            a=None
            b=None
            c=None
            for i in range(targetlen-newminlen):
                if(x1.shape[0]<=newminlen+i):
                    a=np.random.choice(range(self.mindepth,self.maxdepth),1,replace=False)[0]
                if(x1.shape[0]>newminlen+i):
                    a=x1[newminlen+i] 

                if(xs[0].shape[0]<=newminlen+i):
                    b=np.random.choice(range(self.mindepth,self.maxdepth),1,replace=False)[0]
                elif(xs[0].shape[0]>newminlen+i):
                    b=xs[0][newminlen+i]  

                if(xs[1].shape[0]<=newminlen+i):
                    c=np.random.choice(range(self.mindepth,self.maxdepth),1,replace=False)[0]
                elif(xs[1].shape[0]>newminlen+i):
                    c=xs[1][newminlen+i]  
            
                xaa[i]=a + beta * (b - c)
            
            xa = np.concatenate((xa, xaa), axis=None)
            
        for i in range(xa.shape[0]):
            if(xa[i]>self.maxdepth):
                 xa[i]=self.maxdepth
            elif(xa[i]<self.mindepth):
                  xa[i]=self.mindepth
            xa[i] = np.floor(xa[i])
        xa = np.concatenate((np.array(indim,dtype=int),np.array(xa,dtype=int),np.array(self.outdim,dtype=int)), axis=None,dtype=int)
        return xa
    
    def crossoverMean(self,parent,u):
        order = [parent[1:-1],u[1:-1]]
        if(parent.shape[0] > u.shape[0]):
            order = [u[1:-1],parent[1:-1]]
        order[0] = np.resize(order[0],order[1].shape[0])
        middle = np.mean(order,axis=0,dtype=int)
        child=np.insert(middle,0,parent[0])
        child=np.append(child,parent[-1])
        return child.copy()
    
    def crossoverRandomSwap(self,parent,u):
        # the first one is with min len
        order = [parent[1:-1],u[1:-1]]
        child = [parent[0]]
        if(parent.shape[0] > u.shape[0]):
            order = [u[1:-1],parent[1:-1]]
        order[0] = np.resize(order[0],order[1].shape[0])
        swap = np.random.randint(0,2,order[0].shape[0])
 
        for i in range(len(swap)):
            if(swap[i]==0):
                child.append(order[0][i])
            else:
                child.append(order[1][i])
        child.append(parent[-1])
        return np.array(child).copy()
    
    def run(self,beta=0.5):
        current_gen=self.MLPlayerlist
        scores = np.zeros((self.pplSize))
        #initial Run
        #for i in range(len(self.MLPlayerlist)):
        #    b,_,_ = self.fit(self.MLPlayerlist[i],self.stopcount,i)
        #    scores[i]=b
        
        #Generation Run
        for i in range(self.maxiter):
            currentbest = np.min(scores)
            currentmean = np.mean(scores)
            currentbestidx = np.argmin(scores)
            print(f'Run {i:5d} Best: {currentbest}, Mean: {currentmean}, ID:{currentbestidx}, config: {current_gen[currentbestidx]}')
        
            for j in range(self.pplSize):
                parent = current_gen[j]
                midxs = np.random.choice(range(0,self.pplSize),3,replace=False)
                target = self.MLPlayerlist[midxs[2]]
                unitvector = self.mutation_1_2_z(target,self.MLPlayerlist[0:2],beta)
                #print(f'U: {unitvector}')
                #print(f'P: {parent}')
                nextGen = self.crossoverRandomSwap(parent,unitvector)
                print(f'Next Gen: {nextGen}')
                s,_,_ = self.fit(nextGen,self.stopcount,j)
                if(s<scores[j]):
                    scores[j]=s
                    current_gen[j]=nextGen
        
        return 
                

In [95]:
train=np.zeros((4,3,5,6))
val=np.zeros((4,3,5,6))
d = DE_MLP(trainingset=train,validationset=val)

In [96]:
d.run()

Run     0 Best: 0.0, Mean: 0.0, ID:0, config: [90  5  7  8  6  6  5  4  5  6  9  5  5  7  6  5  6  4  5  9  5  6  8  4
  5  4  1]
Next Gen: [90  8  7  8  5  6  5  7  5  6  9  5  5  5  6  5  5  5  5  9  5  5  5  4
  8  4  5  7 17 27  6 23 11 16 13  9  8 34  7 20  5  6 32  5  9  5  6 26
  5  5  4  1]
Next Gen: [90  7  8  7  9  4  7  5  5  6  9  6  7  6  7  5  8  7  6  5  6  5  5  6
  9  5  6  6  7  5  7  7  5  7  5  5  4  6  9  5  7  6  6  5  7  7  8  7
  1]
Next Gen: [90  5  4  5  5  8  9  7  7  7  9  5  5  4  6  8  7  7  7  9  5  1]
Next Gen: [90  5  9  5  5  6  5  8  6  6  6  5  5  6  7  5  5  5  9  6  5  7  5  5
  4  5  8  8 29 18  8 20 25 32 24 31  9  9  4 38 20  7 17 20 17 26  8  8
  4  5  5  6  8  5  9  7  9  5  1]
Next Gen: [90  5  4  7  5  6 40  1]
Next Gen: [90  5  7  5  7  7  5  9  5  7 10  9  9  5  9  7  5  7  5  7  8  5  8  8
  7  4  5  9  5  7  5  4  6  5  7  4  5  9  5  9 10  5  9  5  7  4  5  8
  8  7 10  5  5  5  7 10  4  8  5  1]
Next Gen: [90  4  8  8  4  8  8  5  4  9